# 02 - Préparation des features (Feature Engineering)

Ce notebook implémente le processus complet de préparation des données pour le projet Home Credit Default Risk.

**Objectifs principaux :**
- Charger et fusionner toutes les tables de données
- Créer des features (caractéristiques) pertinentes par agrégation
- Encoder les variables catégorielles
- Préparer le jeu de données final pour la modélisation

**Approche utilisée :**
- Fonction modulaire pour chaque table de données
- Agrégations statistiques (min, max, mean, sum, var) sur les données groupées
- Création de ratios et pourcentages entre variables importantes
- Features spécifiques pour les crédits actifs/fermés et les demandes approuvées/refusées

In [13]:
# Import des bibliothèques nécessaires
import numpy as np
import pandas as pd
import gc  # Garbage collector pour libérer la mémoire
import time
from contextlib import contextmanager
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

print("✓ Bibliothèques importées avec succès")

✓ Bibliothèques importées avec succès


## 1. Fonctions utilitaires

Nous commençons par définir des fonctions helper qui seront utilisées tout au long du notebook.

In [14]:
# Fonction pour mesurer le temps d'exécution
@contextmanager
def timer(title):
    """
    Context manager pour mesurer le temps d'exécution d'un bloc de code.
    Usage: with timer("Mon processus"):
               # code à mesurer
    """
    t0 = time.time()
    yield
    print("{} - terminé en {:.0f}s".format(title, time.time() - t0))

### Encodage One-Hot des variables catégorielles

Le One-Hot encoding transforme les variables catégorielles en colonnes binaires (0 ou 1).
Par exemple, si une colonne "Couleur" contient ["Rouge", "Bleu"], elle sera transformée en deux colonnes : "Couleur_Rouge" et "Couleur_Bleu".

In [15]:
def one_hot_encoder(df, nan_as_category=True):
    """
    Applique le One-Hot encoding aux colonnes catégorielles.
    
    Paramètres:
    -----------
    df : DataFrame
        Le DataFrame à encoder
    nan_as_category : bool
        Si True, les valeurs manquantes (NaN) sont traitées comme une catégorie à part
    
    Retourne:
    ---------
    df : DataFrame encodé
    new_columns : liste des nouvelles colonnes créées
    """
    original_columns = list(df.columns)
    # Identifier les colonnes avec type 'object' (chaînes de caractères = catégorielles)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    # Appliquer pd.get_dummies pour créer les colonnes binaires
    df = pd.get_dummies(df, columns=categorical_columns, dummy_na=nan_as_category)
    # Retourner aussi la liste des nouvelles colonnes créées
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

## 2. Traitement de application_train.csv et application_test.csv

Ces fichiers contiennent les informations principales sur chaque demande de crédit (données du client, montants, etc.).

In [16]:
def application_train_test(num_rows=None, nan_as_category=False):
    """
    Charge et prétraite les données d'application (train + test).
    
    Étapes :
    1. Charge les fichiers train et test
    2. Fusionne les deux datasets
    3. Nettoie les données (suppression de valeurs aberrantes)
    4. Encode les variables catégorielles
    5. Crée de nouvelles features (ratios, pourcentages)
    """
    # Chargement des données
    df = pd.read_csv('../data/raw/application_train.csv', nrows=num_rows)
    test_df = pd.read_csv('../data/raw/application_test.csv', nrows=num_rows)
    print("Échantillons train: {}, test: {}".format(len(df), len(test_df)))
    
    # Fusionner train et test pour appliquer les mêmes transformations
    # Note: Utiliser pd.concat() au lieu de .append() (deprecated dans pandas 2.0+)
    df = pd.concat([df, test_df], ignore_index=True)
    
    # Nettoyage : Supprimer les 4 applications avec CODE_GENDER = 'XNA' (valeur aberrante)
    df = df[df['CODE_GENDER'] != 'XNA']
    
    # Encodage binaire (0 ou 1) pour les features avec seulement 2 catégories
    for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
        df[bin_feature], uniques = pd.factorize(df[bin_feature])
    
    # One-Hot encoding pour les autres features catégorielles
    df, cat_cols = one_hot_encoder(df, nan_as_category)
    
    # Nettoyage : La valeur 365243 pour DAYS_EMPLOYED est une valeur sentinel (code pour "inconnu")
    # On la remplace par NaN
    df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)
    
    # Création de nouvelles features (ratios et pourcentages)
    # Ces ratios sont souvent plus informatifs que les valeurs absolues
    
    # Pourcentage d'emploi par rapport à l'âge
    df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    
    # Pourcentage du crédit par rapport au revenu
    df['INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
    
    # Revenu par personne dans le foyer
    df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
    
    # Pourcentage de l'annuité par rapport au revenu (capacité de remboursement)
    df['ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    
    # Taux de paiement : annuité / montant du crédit
    df['PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']
    
    # Libération de la mémoire
    del test_df
    gc.collect()
    
    return df

## 3. Traitement de bureau.csv et bureau_balance.csv

**bureau.csv** : Historique des crédits antérieurs du client auprès d'autres institutions financières  
**bureau_balance.csv** : Historique mensuel des soldes pour ces crédits bureau

**Stratégie :**
- Agréger bureau_balance au niveau bureau (une ligne par crédit)
- Créer des features distinctes pour les crédits ACTIFS vs FERMÉS
- Agréger au niveau client (SK_ID_CURR)

In [17]:
def bureau_and_balance(num_rows=None, nan_as_category=True):
    """
    Traite les données bureau (crédits externes du client).
    
    Étapes :
    1. Charge bureau et bureau_balance
    2. Agrège bureau_balance par crédit (SK_ID_BUREAU)
    3. Fusionne avec bureau
    4. Crée des agrégations générales par client
    5. Crée des features spécifiques pour crédits actifs
    6. Crée des features spécifiques pour crédits fermés
    """
    # Chargement des données
    bureau = pd.read_csv('../data/raw/bureau.csv', nrows=num_rows)
    bb = pd.read_csv('../data/raw/bureau_balance.csv', nrows=num_rows)
    
    # Encodage des variables catégorielles
    bb, bb_cat = one_hot_encoder(bb, nan_as_category)
    bureau, bureau_cat = one_hot_encoder(bureau, nan_as_category)
    
    # === BUREAU BALANCE : Agrégation au niveau crédit ===
    # Pour chaque crédit (SK_ID_BUREAU), on calcule des statistiques sur les mois
    bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}
    # Pour chaque colonne catégorielle encodée, on calcule la moyenne
    for col in bb_cat:
        bb_aggregations[col] = ['mean']
    
    bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)
    # Renommer les colonnes pour indiquer la provenance
    bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])
    
    # Joindre les agrégations de bureau_balance à bureau
    bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')
    bureau.drop(['SK_ID_BUREAU'], axis=1, inplace=True)
    
    del bb, bb_agg
    gc.collect()
    
    # === BUREAU : Agrégations numériques ===
    # Définir les agrégations à calculer pour chaque feature numérique
    num_aggregations = {
        'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
        'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
        'DAYS_CREDIT_UPDATE': ['mean'],
        'CREDIT_DAY_OVERDUE': ['max', 'mean'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        'AMT_ANNUITY': ['max', 'mean'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'MONTHS_BALANCE_MIN': ['min'],
        'MONTHS_BALANCE_MAX': ['max'],
        'MONTHS_BALANCE_SIZE': ['mean', 'sum']
    }
    
    # === BUREAU : Agrégations catégorielles ===
    cat_aggregations = {}
    for cat in bureau_cat:
        cat_aggregations[cat] = ['mean']
    for cat in bb_cat:
        cat_aggregations[cat + "_MEAN"] = ['mean']
    
    # Agrégation générale par client (SK_ID_CURR)
    bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])
    
    # === CRÉDITS ACTIFS : Features spécifiques ===
    # Filtrer uniquement les crédits actifs et créer des agrégations spécifiques
    active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
    active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
    active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(active_agg, how='left', on='SK_ID_CURR')
    
    del active, active_agg
    gc.collect()
    
    # === CRÉDITS FERMÉS : Features spécifiques ===
    # Même logique pour les crédits fermés
    closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
    closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
    closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(closed_agg, how='left', on='SK_ID_CURR')
    
    del closed, closed_agg, bureau
    gc.collect()
    
    return bureau_agg

## 4. Traitement de previous_application.csv

Ce fichier contient toutes les demandes de crédit précédentes du client chez Home Credit.

**Stratégie :**
- Créer des agrégations générales
- Features spécifiques pour demandes APPROUVÉES
- Features spécifiques pour demandes REFUSÉES

In [18]:
def previous_applications(num_rows=None, nan_as_category=True):
    """
    Traite les demandes de crédit précédentes.
    
    Étapes :
    1. Charge previous_application
    2. Nettoie les valeurs sentinelles (365243 = inconnu)
    3. Crée de nouvelles features (ratios)
    4. Agrégations générales par client
    5. Features spécifiques pour demandes approuvées
    6. Features spécifiques pour demandes refusées
    """
    # Chargement des données
    prev = pd.read_csv('../data/raw/previous_application.csv', nrows=num_rows)
    prev, cat_cols = one_hot_encoder(prev, nan_as_category=True)
    
    # Nettoyage : Remplacer les valeurs sentinel 365243 par NaN
    prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace=True)
    prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace=True)
    prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace=True)
    prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace=True)
    prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace=True)
    
    # Nouvelle feature : Pourcentage entre montant demandé et montant reçu
    # Indique si le client a obtenu ce qu'il demandait
    prev['APP_CREDIT_PERC'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']
    
    # === Agrégations numériques ===
    num_aggregations = {
        'AMT_ANNUITY': ['min', 'max', 'mean'],
        'AMT_APPLICATION': ['min', 'max', 'mean'],
        'AMT_CREDIT': ['min', 'max', 'mean'],
        'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
        'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum'],
    }
    
    # === Agrégations catégorielles ===
    cat_aggregations = {}
    for cat in cat_cols:
        cat_aggregations[cat] = ['mean']
    
    # Agrégation générale par client
    prev_agg = prev.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])
    
    # === DEMANDES APPROUVÉES : Features spécifiques ===
    approved = prev[prev['NAME_CONTRACT_STATUS_Approved'] == 1]
    approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
    approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
    prev_agg = prev_agg.join(approved_agg, how='left', on='SK_ID_CURR')
    
    # === DEMANDES REFUSÉES : Features spécifiques ===
    refused = prev[prev['NAME_CONTRACT_STATUS_Refused'] == 1]
    refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
    refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
    prev_agg = prev_agg.join(refused_agg, how='left', on='SK_ID_CURR')
    
    del refused, refused_agg, approved, approved_agg, prev
    gc.collect()
    
    return prev_agg

## 5. Traitement de POS_CASH_balance.csv

Ce fichier contient les historiques mensuels des soldes pour les crédits POS (Point of Sale) et CASH.

In [19]:
def pos_cash(num_rows=None, nan_as_category=True):
    """
    Traite les données de soldes POS et CASH.
    
    Agrège les informations mensuelles au niveau client :
    - Nombre de mois d'historique
    - Retards de paiement (DPD = Days Past Due)
    - Distribution des statuts de paiement
    """
    # Chargement des données
    pos = pd.read_csv('../data/raw/POS_CASH_balance.csv', nrows=num_rows)
    pos, cat_cols = one_hot_encoder(pos, nan_as_category=True)
    
    # === Agrégations ===
    aggregations = {
        'MONTHS_BALANCE': ['max', 'mean', 'size'],  # size = nombre de mois
        'SK_DPD': ['max', 'mean'],                   # Jours de retard
        'SK_DPD_DEF': ['max', 'mean']                # Jours de retard (définition alternative)
    }
    
    # Agrégations pour les colonnes catégorielles
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    
    pos_agg = pos.groupby('SK_ID_CURR').agg(aggregations)
    pos_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])
    
    # Compter le nombre de comptes POS CASH pour chaque client
    pos_agg['POS_COUNT'] = pos.groupby('SK_ID_CURR').size()
    
    del pos
    gc.collect()
    
    return pos_agg

## 6. Traitement de installments_payments.csv

Ce fichier contient l'historique de remboursement des versements précédents (installments).  
**Idée clé :** Comparer ce qui devait être payé (AMT_INSTALMENT) avec ce qui a réellement été payé (AMT_PAYMENT).

In [20]:
def installments_payments(num_rows=None, nan_as_category=True):
    """
    Traite l'historique des paiements par versements.
    
    Crée des features pour mesurer le comportement de paiement :
    - DPD : Days Past Due (jours de retard)
    - DBD : Days Before Due (jours d'avance)
    - PAYMENT_PERC : Pourcentage payé vs attendu
    - PAYMENT_DIFF : Différence entre attendu et payé
    """
    # Chargement des données
    ins = pd.read_csv('../data/raw/installments_payments.csv', nrows=num_rows)
    ins, cat_cols = one_hot_encoder(ins, nan_as_category=True)
    
    # === Nouvelles features de comportement de paiement ===
    
    # Pourcentage payé par rapport au montant prévu
    ins['PAYMENT_PERC'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
    
    # Différence entre montant prévu et montant payé (positif = sous-paiement)
    ins['PAYMENT_DIFF'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT']
    
    # DPD : Days Past Due = nombre de jours de retard (seulement valeurs positives)
    ins['DPD'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
    ins['DPD'] = ins['DPD'].apply(lambda x: x if x > 0 else 0)
    
    # DBD : Days Before Due = nombre de jours d'avance (seulement valeurs positives)
    ins['DBD'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']
    ins['DBD'] = ins['DBD'].apply(lambda x: x if x > 0 else 0)
    
    # === Agrégations ===
    aggregations = {
        'NUM_INSTALMENT_VERSION': ['nunique'],      # Nombre de versions différentes
        'DPD': ['max', 'mean', 'sum'],              # Statistiques sur les retards
        'DBD': ['max', 'mean', 'sum'],              # Statistiques sur les avances
        'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],  # Comportement de paiement
        'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
        'AMT_INSTALMENT': ['max', 'mean', 'sum'],
        'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
        'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']
    }
    
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    
    ins_agg = ins.groupby('SK_ID_CURR').agg(aggregations)
    ins_agg.columns = pd.Index(['INSTAL_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])
    
    # Compter le nombre de versements pour chaque client
    ins_agg['INSTAL_COUNT'] = ins.groupby('SK_ID_CURR').size()
    
    del ins
    gc.collect()
    
    return ins_agg

## 7. Traitement de credit_card_balance.csv

Ce fichier contient les historiques mensuels des soldes de cartes de crédit.

In [21]:
def credit_card_balance(num_rows=None, nan_as_category=True):
    """
    Traite les données de soldes de cartes de crédit.
    
    Applique des agrégations générales (min, max, mean, sum, var) 
    sur toutes les colonnes numériques.
    """
    # Chargement des données
    cc = pd.read_csv('../data/raw/credit_card_balance.csv', nrows=num_rows)
    cc, cat_cols = one_hot_encoder(cc, nan_as_category=True)
    
    # On n'a pas besoin de SK_ID_PREV pour les agrégations finales
    cc.drop(['SK_ID_PREV'], axis=1, inplace=True)
    
    # === Agrégations générales ===
    # Pour chaque colonne, on calcule min, max, mean, sum et variance
    cc_agg = cc.groupby('SK_ID_CURR').agg(['min', 'max', 'mean', 'sum', 'var'])
    cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()])
    
    # Compter le nombre de lignes (mois) de carte de crédit par client
    cc_agg['CC_COUNT'] = cc.groupby('SK_ID_CURR').size()
    
    del cc
    gc.collect()
    
    return cc_agg

## 8. Fonction principale : Fusion de toutes les données

Cette fonction orchestre tout le processus :
1. Charge et traite les données principales (application)
2. Charge et fusionne chaque table secondaire
3. Retourne le DataFrame final prêt pour la modélisation

In [22]:
def prepare_full_dataset(debug=False):
    """
    Fonction principale qui orchestre toute la préparation des données.
    
    Paramètres:
    -----------
    debug : bool
        Si True, charge seulement 10000 lignes de chaque fichier (pour tests rapides)
    
    Retourne:
    ---------
    df : DataFrame complet avec toutes les features
    """
    # En mode debug, on limite le nombre de lignes pour aller plus vite
    num_rows = 10000 if debug else None
    
    # === 1. Charger les données principales ===
    print("\n" + "="*80)
    print("ÉTAPE 1 : Chargement des données application (train + test)")
    print("="*80)
    df = application_train_test(num_rows)
    print(f"✓ Shape après application : {df.shape}")
    
    # === 2. Bureau et bureau_balance ===
    print("\n" + "="*80)
    print("ÉTAPE 2 : Traitement des données Bureau (crédits externes)")
    print("="*80)
    with timer("Traitement bureau et bureau_balance"):
        bureau = bureau_and_balance(num_rows)
        print(f"   Bureau shape: {bureau.shape}")
        df = df.join(bureau, how='left', on='SK_ID_CURR')
        del bureau
        gc.collect()
    print(f"✓ Shape après fusion bureau : {df.shape}")
    
    # === 3. Previous applications ===
    print("\n" + "="*80)
    print("ÉTAPE 3 : Traitement des demandes précédentes")
    print("="*80)
    with timer("Traitement previous_applications"):
        prev = previous_applications(num_rows)
        print(f"   Previous applications shape: {prev.shape}")
        df = df.join(prev, how='left', on='SK_ID_CURR')
        del prev
        gc.collect()
    print(f"✓ Shape après fusion previous : {df.shape}")
    
    # === 4. POS-CASH balance ===
    print("\n" + "="*80)
    print("ÉTAPE 4 : Traitement des soldes POS-CASH")
    print("="*80)
    with timer("Traitement POS-CASH balance"):
        pos = pos_cash(num_rows)
        print(f"   Pos-cash balance shape: {pos.shape}")
        df = df.join(pos, how='left', on='SK_ID_CURR')
        del pos
        gc.collect()
    print(f"✓ Shape après fusion POS : {df.shape}")
    
    # === 5. Installments payments ===
    print("\n" + "="*80)
    print("ÉTAPE 5 : Traitement des paiements par versements")
    print("="*80)
    with timer("Traitement installments payments"):
        ins = installments_payments(num_rows)
        print(f"   Installments payments shape: {ins.shape}")
        df = df.join(ins, how='left', on='SK_ID_CURR')
        del ins
        gc.collect()
    print(f"✓ Shape après fusion installments : {df.shape}")
    
    # === 6. Credit card balance ===
    print("\n" + "="*80)
    print("ÉTAPE 6 : Traitement des soldes de cartes de crédit")
    print("="*80)
    with timer("Traitement credit card balance"):
        cc = credit_card_balance(num_rows)
        print(f"   Credit card balance shape: {cc.shape}")
        df = df.join(cc, how='left', on='SK_ID_CURR')
        del cc
        gc.collect()
    print(f"✓ Shape après fusion credit card : {df.shape}")
    
    print("\n" + "="*80)
    print("PRÉPARATION TERMINÉE !")
    print("="*80)
    print(f"Dataset final : {df.shape[0]} lignes, {df.shape[1]} colonnes")
    
    return df

## 9. Exécution du pipeline de préparation

Maintenant, exécutons le pipeline complet pour préparer nos données.

In [23]:
# Exécuter en mode DEBUG (10000 lignes) pour un test rapide
# Pour la version complète, mettre debug=False
DEBUG_MODE = True

print("🚀 Début de la préparation des données...")
print(f"Mode: {'DEBUG (10000 lignes)' if DEBUG_MODE else 'COMPLET'}\n")

with timer("Pipeline complet de préparation"):
    df_final = prepare_full_dataset(debug=DEBUG_MODE)

🚀 Début de la préparation des données...
Mode: DEBUG (10000 lignes)


ÉTAPE 1 : Chargement des données application (train + test)
Échantillons train: 10000, test: 10000
✓ Shape après application : (20000, 245)

ÉTAPE 2 : Traitement des données Bureau (crédits externes)
   Bureau shape: (2011, 108)
Traitement bureau et bureau_balance - terminé en 0s
✓ Shape après fusion bureau : (20000, 353)

ÉTAPE 3 : Traitement des demandes précédentes
   Previous applications shape: (9734, 242)
Traitement previous_applications - terminé en 0s
✓ Shape après fusion previous : (20000, 595)

ÉTAPE 4 : Traitement des soldes POS-CASH
   Pos-cash balance shape: (9494, 15)
Traitement POS-CASH balance - terminé en 0s
✓ Shape après fusion POS : (20000, 610)

ÉTAPE 5 : Traitement des paiements par versements
   Installments payments shape: (8893, 26)
Traitement installments payments - terminé en 0s
✓ Shape après fusion installments : (20000, 636)

ÉTAPE 6 : Traitement des soldes de cartes de crédit
   Credit ca

## 10. Exploration du dataset final

Examinons le résultat de notre préparation.

In [24]:
# Aperçu général du dataset
print("📊 APERÇU DU DATASET FINAL")
print("="*80)
print(f"Nombre de lignes : {df_final.shape[0]:,}")
print(f"Nombre de colonnes (features) : {df_final.shape[1]:,}")
print(f"\nMémoire utilisée : {df_final.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print("\nPremières colonnes :")
print(df_final.columns.tolist()[:20])

📊 APERÇU DU DATASET FINAL
Nombre de lignes : 20,000
Nombre de colonnes (features) : 767

Mémoire utilisée : 105.07 MB

Premières colonnes :
['SK_ID_CURR', 'TARGET', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE']


### Séparation train/test

In [25]:
# Séparer les données train (avec TARGET) et test (sans TARGET)
train_df = df_final[df_final['TARGET'].notnull()].copy()
test_df = df_final[df_final['TARGET'].isnull()].copy()

print("📊 SÉPARATION TRAIN / TEST")
print("="*80)
print(f"Train shape : {train_df.shape}")
print(f"Test shape  : {test_df.shape}")
print(f"\nDistribution de la variable cible (TARGET) dans train :")
print(train_df['TARGET'].value_counts())
print(f"\nPourcentage de défaut : {train_df['TARGET'].mean()*100:.2f}%")

📊 SÉPARATION TRAIN / TEST
Train shape : (10000, 767)
Test shape  : (10000, 767)

Distribution de la variable cible (TARGET) dans train :
TARGET
0.0    9225
1.0     775
Name: count, dtype: int64

Pourcentage de défaut : 7.75%


### Analyse des valeurs manquantes

In [26]:
# Calculer le pourcentage de valeurs manquantes par colonne
missing_values = train_df.isnull().sum()
missing_percent = (missing_values / len(train_df)) * 100
missing_df = pd.DataFrame({
    'Colonne': missing_values.index,
    'Valeurs_manquantes': missing_values.values,
    'Pourcentage': missing_percent.values
})

# Filtrer les colonnes avec au moins 1% de valeurs manquantes
missing_df = missing_df[missing_df['Pourcentage'] > 1].sort_values('Pourcentage', ascending=False)

print("📊 COLONNES AVEC VALEURS MANQUANTES (>1%)")
print("="*80)
print(f"Nombre de colonnes concernées : {len(missing_df)}")
print("\nTop 10 colonnes avec le plus de valeurs manquantes :")
print(missing_df.head(10).to_string(index=False))

📊 COLONNES AVEC VALEURS MANQUANTES (>1%)
Nombre de colonnes concernées : 576

Top 10 colonnes avec le plus de valeurs manquantes :
                      Colonne  Valeurs_manquantes  Pourcentage
  BURO_MONTHS_BALANCE_MIN_MIN               10000        100.0
  BURO_MONTHS_BALANCE_MAX_MAX               10000        100.0
BURO_MONTHS_BALANCE_SIZE_MEAN               10000        100.0
      BURO_STATUS_C_MEAN_MEAN               10000        100.0
      BURO_STATUS_3_MEAN_MEAN               10000        100.0
    BURO_STATUS_nan_MEAN_MEAN               10000        100.0
      BURO_STATUS_X_MEAN_MEAN               10000        100.0
  REFUSED_APP_CREDIT_PERC_VAR               10000        100.0
      BURO_STATUS_1_MEAN_MEAN               10000        100.0
      BURO_STATUS_2_MEAN_MEAN               10000        100.0


### Aperçu des features créées par catégorie

In [27]:
# Compter les features par préfixe (provenance)
prefixes = {
    'Application': [col for col in df_final.columns if not any(col.startswith(p) for p in ['BURO_', 'ACTIVE_', 'CLOSED_', 'PREV_', 'APPROVED_', 'REFUSED_', 'POS_', 'INSTAL_', 'CC_'])],
    'Bureau (BURO)': [col for col in df_final.columns if col.startswith('BURO_')],
    'Bureau Active': [col for col in df_final.columns if col.startswith('ACTIVE_')],
    'Bureau Closed': [col for col in df_final.columns if col.startswith('CLOSED_')],
    'Previous (PREV)': [col for col in df_final.columns if col.startswith('PREV_')],
    'Approved': [col for col in df_final.columns if col.startswith('APPROVED_')],
    'Refused': [col for col in df_final.columns if col.startswith('REFUSED_')],
    'POS Cash': [col for col in df_final.columns if col.startswith('POS_')],
    'Installments': [col for col in df_final.columns if col.startswith('INSTAL_')],
    'Credit Card': [col for col in df_final.columns if col.startswith('CC_')]
}

print("📊 RÉPARTITION DES FEATURES PAR ORIGINE")
print("="*80)
for name, cols in prefixes.items():
    print(f"{name:20s} : {len(cols):4d} features")
    
print(f"\n{'TOTAL':20s} : {df_final.shape[1]:4d} features")

📊 RÉPARTITION DES FEATURES PAR ORIGINE
Application          :  245 features
Bureau (BURO)        :   54 features
Bureau Active        :   27 features
Bureau Closed        :   27 features
Previous (PREV)      :  182 features
Approved             :   30 features
Refused              :   30 features
POS Cash             :   15 features
Installments         :   26 features
Credit Card          :  131 features

TOTAL                :  767 features


## 11. Sauvegarde des données préparées

Sauvegardons nos datasets préparés pour une utilisation ultérieure dans la modélisation.

In [28]:
# Créer le répertoire de sortie s'il n'existe pas
import os
os.makedirs('../data/processed', exist_ok=True)

# Sauvegarder le dataset complet
output_path_full = '../data/processed/features_full.csv'
df_final.to_csv(output_path_full, index=False)
print(f"✓ Dataset complet sauvegardé : {output_path_full}")
print(f"  Taille du fichier : {os.path.getsize(output_path_full) / 1024**2:.2f} MB")

# Sauvegarder séparément train et test
output_path_train = '../data/processed/features_train.csv'
output_path_test = '../data/processed/features_test.csv'

train_df.to_csv(output_path_train, index=False)
test_df.to_csv(output_path_test, index=False)

print(f"\n✓ Train sauvegardé : {output_path_train}")
print(f"  Taille du fichier : {os.path.getsize(output_path_train) / 1024**2:.2f} MB")
print(f"\n✓ Test sauvegardé : {output_path_test}")
print(f"  Taille du fichier : {os.path.getsize(output_path_test) / 1024**2:.2f} MB")

✓ Dataset complet sauvegardé : ../data/processed/features_full.csv
  Taille du fichier : 34.77 MB

✓ Train sauvegardé : ../data/processed/features_train.csv
  Taille du fichier : 17.30 MB

✓ Test sauvegardé : ../data/processed/features_test.csv
  Taille du fichier : 17.49 MB


## 12. Résumé et prochaines étapes

### ✅ Ce qui a été fait dans ce notebook :

1. **Chargement et fusion** de 7 tables de données différentes
2. **Nettoyage** des valeurs aberrantes et sentinelles (365243 → NaN)
3. **Encodage** des variables catégorielles (One-Hot encoding)
4. **Création de features** par agrégation (min, max, mean, sum, var)
5. **Features spécifiques** :
   - Ratios et pourcentages (ex: INCOME_CREDIT_PERC, PAYMENT_RATE)
   - Comportement de paiement (DPD, DBD, PAYMENT_PERC)
   - Distinction crédits actifs/fermés
   - Distinction demandes approuvées/refusées
6. **Séparation** train/test
7. **Sauvegarde** des données préparées

### 📊 Résultat :

- **Dataset final** : ~{df_final.shape[1]} features créées
- **Prêt pour la modélisation** avec LightGBM ou autre algorithme

### 🔜 Prochaines étapes :

1. **Feature Selection** : Identifier les features les plus importantes
2. **Modélisation** : Entraîner un modèle LightGBM avec validation croisée
3. **Optimisation** : Tuning des hyperparamètres
4. **Évaluation** : Analyser les performances (ROC-AUC)
5. **Prédictions** : Générer les prédictions pour le test set

---

**Note importante** : Ce notebook utilise l'approche du kernel Kaggle "LightGBM with Simple Features" de jsaguiar, qui a obtenu d'excellents résultats sur cette compétition. L'approche privilégie la création de nombreuses features par agrégation, ce qui peut entraîner de l'overfitting. Une sélection de features sera donc importante dans les étapes suivantes.